<img src="logo.png">

# Ajuste de distribuciones


El **ajuste de distribuciones** se refiere a aplicar una o varias técnicas gráficas y analíticas para tener una idea aproximada sobre el tipo de variable aleatoria al que pertencen tus datos.

Por ejemplo, si tus datos son de tipo discreto (como el año de nacimiento o el número de delitos cometidos mes a mes), entonces vale la pena preguntarse si tus datos provienen de una distribución de Bernoulli, binomial, poisson, geométrica, etc.

Y si tus datos son de tipo continuo (como la velocidad de crecimiento de una población o el tiempo de espera entre la llegada de dos clientes a un restaurante), entonces nos preguntamos si los datos provienen de una gaussiana, una exponencial, una uniforme, etc.

**Ejemplo.** 

Supongamos que tienes los datos del dataframe [ejemplo01](https://github.com/scidatmath2020/Inferencia-Estad-stica-2022/blob/main/data/ejemplo_01.csv).

Digamos que esos datos provienen de una variable aleatoria $X$. Es decir, tomamos una muestra de tamaño 100 de cierta población de estudio y hemos tomado la medida de cierta característica $X$ de esa población.

Mostramos el histograma junto con la densidad de esta muestra. A la densidad mostrada se le conoce como **densidad empírica.**

<img src="im006.png">

Uno bien puede preguntarse, ¿cuál es la probabilidad de que el valor de la característica $X$ de un individuo tomado al azar sea menor a 2.25?

O en otras palabras, **¿qué porcentaje de los individuos de la población cumplen que el valor de su característica $X$ es menor que 2.25?**

Sabemos que el histograma nos indica que mida las alturas de cada uno de los primeros 5 rectángulos. Sin embargo, **recuerda que ese histograma únicamente corresponde con una muestra de tamaño 100 de nuestra población**, por lo cual la respuesta dada por este método de medir alturas no tiene por qué ser buena. 

Pero peor aún: **el uso de densidad empírica conlleva un sobreajuste de los datos**. Por ejemplo, si los datos realmente provienen de una gaussiana pero tienes varios atípicos, la densidad empírica puede no parecerse a una gaussiana.

Es por ello que necesitamos establecer la verdadera densidad a la que se ajusten casi todas las muestras posibles. Esta densidad es lo que se conoce como **densidad teórica**.

De esta manera, podemos decir que el ajuste de distribuciones son las técnicas estadísticas que permiten, a través de las densidades empíricas, conocer aproximadamente a la densidad teórica.

## Proceso para ajustar distribuciones

Básicamente se aplican tres pasos:

1. Observar la forma de los datos (gráficas).

2. Ajustar los datos con los tipos de distribuciones que tu intuición te indique basado en el paso anterior.

3. Verificar cuál de los modelos se ajustó mejor.

Ahora bien, es importante tomar en cuenta que **no todos los datos corresponden a una distribución "famosa" o sencilla**. El proceso de ajuste de distribuciones es más un arte que una ciencia. Este proceso depende mucho de la interpretación del investigador así como de su experiencia y **perseverancia** en el sentido de tener que buscar y buscar entre las distribuciones posibles hasta encontrar una que se ajuste al modelo.

En **R** contamos con la paquetería ``fitdistrplus`` (y muchas otras como ``MASS``) que nos ayudan a realizar ajustes de distribuciones.

**Ejemplo.**

Consideremos la tabla [ejemplo_02.csv](https://github.com/scidatmath2020/Inferencia-Estad-stica-2022/blob/main/data/ejemplo_02.csv).

**1. Observar la forma de los datos**

De la paquetería ``fitdistrplus`` utilizar la función ``descdist()`` cuya sintaxis es ``descdist(datos,discrete=FALSE,boot)`` donde

``datos:`` es tu vector de datos.

``discrete:`` FALSE si tus datos son de tipo continuo; TRUE si son de tipo discreto.

``boot:`` número de muestreos que se van a tomar. Debe ser un valor mayor a 10.

El resultado es cierta información estadística junto con una gráfica de tipo Cullen y Frey. El resultado de la gráfica indica posibles candidatos a la distribución teórica.

En el caso de **ejemplo_02.csv**, obtenemos gamma, lognormal y exponencial:

``descdist(ejemplo_02,discrete=FALSE,boot=500)``

<img src="im007.png">


**2. Ajustar los datos con los tipos de distribuciones que tu intuición te indique basado en el paso anterior**

Utilizando la función ``fitdist()`` de la paquetería ``fitdistrplus``, realizamos el ajuste para la distribución teórica candidata. La sintaxis de ``fitdist()`` es ``fitdist(datos,tipo)`` donde

``datos:`` es tu vector de datos.

``tipo:`` es un tipo de distribución, o un vector con los nombres de las distribuciones candidatas, las cuales propones basado en el paso 1. Estas pueden ser del tipo "norm", "lnorm", "pois", "exp", "gamma", "nbinom", "geom", "beta", "unif".

En el caso del **ejemplo_02.csv**, proponemos gamma, lognormal y exp.

``mis_distribuciones <- c("gamma","lnorm","exp")``

``ajustes <- lapply(mis_distribuciones,function(x){fitdist(ejemplo_02,x)})``

En este caso, ``ajustes`` es una lista con los resultados de los tres modelos elegidos.

**3. Verificar cuál de los modelos se ajustó mejor.**

Ya que hemos aplicado la función ``fitdist()`` a nuestros datos con cada uno de los modelos, el resultado de cada modelo es un objeto al que se le pueden aplicar tres (de hecho más) funciones importantes: ``plot()``, ``summary()`` y ``coef()``

A partir del ``plot()`` podemos ir descartando candidatos. Y para finalizar, utilizamos alguno de los siguientes criterios:

* Elegir el que mayor **loglike** tenga.

* Elegir el que menor **aic** tenga (criterio de información de Akaike). O

* Elegir el que menor **bic** tenga (criterio de información de Bayes).

Del **ejemplo_02.csv**, haciendo ``plot(ajustes[[3]])`` (es decir, del modelo exponencial), vemos que el gráfico Q-Q es muy malo, por lo cual descartamos este modelo.

Para decidir cuál de los modelos restantes se ajusta mejor (gamma o lognormal), podemos ver que el aic del gamma (que se obtiene como ``ajustes[[1]]$aic`` es mayor que el de la lognormal (que se obtiene con ``ajustes[[2]]$aic``)

Por lo tanto, concluímos que la población (y no solo la muestra) es una lognormal. Para obtener sus parámetros, hacemos ``coef(ajustes[[2]])``. La conclusión se puede informar de la siguiente manera: "de entre todas las distribuciones testeadas, la lognormal con media 2.68 y desviación 0.575 es la que menor información de Akaike presenta, por lo cual es la que mejor se ajusta a nuestros datos". 

**Observación.** La data **ejemplo_02.csv** fue de hecho tomada de una lognormal con media 2.7 y desviación 0.6.